# Sentinel 2 Collection

In [1]:
from geetools import collection, tools, ui

In [2]:
import ee

This means that the default collection for Landsat 1 is RAW and MSS, for Landsat 5 is SR and TM

# HELPERS

In [3]:
def print_values(image):
    print(tools.image.get_value(image, image.geometry().centroid(), 30, side='client'))

## Sentinel 2 SR

In [4]:
s2sr = collection.Sentinel2SR

In [5]:
collection.info(s2sr)

{'SCL': {1: 'saturated',
  2: 'dark',
  3: 'shadow',
  4: 'vegetation',
  5: 'bare_soil',
  6: 'water',
  7: 'cloud_low',
  8: 'cloud_medium',
  9: 'cloud_high',
  10: 'cirrus',
  11: 'snow'},
 'algorithms': {'scl_masks': <bound method Sentinel2.SCL_masks of <geetools.collection.sentinel.Sentinel2 object at 0x7fe043d20f28>>},
 'band_names': ['aerosol',
  'blue',
  'green',
  'red',
  'red_edge_1',
  'red_edge_2',
  'red_edge_3',
  'nir',
  'red_edge_4',
  'water_vapor',
  'swir',
  'swir2',
  'aerosol_thickness',
  'water_vapor_pressure',
  'scene_classification_map'],
 'bands': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B8A',
  'B9',
  'B11',
  'B12',
  'AOT',
  'WVP',
  'SCL'],
 'bit_bands': [],
 'classification_bands': ['SCL'],
 'cloud_cover': 'CLOUD_COVERAGE_ASSESSMENT',
 'ee_collection': <ee.imagecollection.ImageCollection at 0x7fe043d1c7b8>,
 'end_date': '2019-04-09',
 'id': 'COPERNICUS/S2_SR',
 'indices': {'evi': <function geetools.collection.Collection.ev

In [6]:
s2sr_i = ee.Image(s2sr.collection.first())

## Rename

In [9]:
s2sr_renamed = s2sr.rename(s2sr_i)

In [10]:
ui.eprint(s2sr_renamed.bandNames())

[ 'aerosol',
  'blue',
  'green',
  'red',
  'red_edge_1',
  'red_edge_2',
  'red_edge_3',
  'nir',
  'red_edge_4',
  'water_vapor',
  'swir',
  'swir2',
  'aerosol_thickness',
  'water_vapor_pressure',
  'scene_classification_map',
  'TCI_R',
  'TCI_G',
  'TCI_B',
  'MSK_CLDPRB',
  'MSK_SNWPRB',
  'QA10',
  'QA20',
  'QA60']



## Rename collection

In [38]:
s2col = s2sr.collection

In [39]:
s2col_renamed = s2col.map(lambda img: s2sr.rename(img))

In [40]:
tools.imagecollection.get_values(s2col_renamed.limit(2), ee.Image(s2col.first()).geometry().centroid(), 20, side='client')

{'20190224T000241_20190224T000236_T55GCM': {'MSK_CLDPRB': 0.0,
  'MSK_SNWPRB': 0.0,
  'QA10': 0.0,
  'QA20': 0.0,
  'QA60': 0.0,
  'TCI_B': 20.0,
  'TCI_G': 17.0,
  'TCI_R': 10.0,
  'aerosol': 238.0,
  'aerosol_thickness': 204.0,
  'blue': 186.0,
  'green': 159.0,
  'nir': 77.0,
  'red': 97.0,
  'red_edge_1': 107.0,
  'red_edge_2': 99.0,
  'red_edge_3': 90.0,
  'red_edge_4': 97.0,
  'scene_classification_map': 6.0,
  'swir': 62.0,
  'swir2': 45.0,
  'water_vapor': 40.0,
  'water_vapor_pressure': 1152.0},
 '20190224T000241_20190224T000236_T55GCN': {'MSK_CLDPRB': None,
  'MSK_SNWPRB': None,
  'QA10': None,
  'QA20': None,
  'QA60': None,
  'TCI_B': None,
  'TCI_G': None,
  'TCI_R': None,
  'aerosol': None,
  'aerosol_thickness': None,
  'blue': None,
  'green': None,
  'nir': None,
  'red': None,
  'red_edge_1': None,
  'red_edge_2': None,
  'red_edge_3': None,
  'red_edge_4': None,
  'scene_classification_map': None,
  'swir': None,
  'swir2': None,
  'water_vapor': None,
  'water_vapor

## SCL Mask

In [41]:
mask = s2sr.SCL_masks(s2sr_i)

## Visualization

In [15]:
Map = ui.Map()

In [16]:
Map.show()

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

### renamed image

In [17]:
p = ee.Geometry.Point([-71, -41])

In [42]:
s2col_renamed = s2col_renamed.filterBounds(p)

In [43]:
s2sr_renamed = s2col_renamed.first()

In [45]:
vis = s2sr.visualization('NSR', True)

In [46]:
Map.addLayer(s2sr_renamed, vis, 'S2 SR Renamed NSR')

In [22]:
Map.centerObject(s2sr_renamed)

## SLC

In [47]:
visSLC = s2sr.visualization('SCL', True)

In [ ]:
Map.addLayer(s2sr_renamed, visSLC)